In [1]:
import sklearn
sklearn.__version__

'1.0.1'

In [1]:
import pandas as pd
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

file_path= 'C:/data/titanic.csv'

df=pd.read_csv(file_path)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
del df['PassengerId']
del df['Name']
del df['Ticket']
del df['Cabin']

df.dropna(thresh=int(len(df) * 0.5), axis=1)
df['Embarked']= df['Embarked'].fillna('S')
df['Age']= df['Age'].fillna(df['Age'].mean())

In [3]:
df['Sex']=df['Sex'].astype('category')
df['Pclass']=df['Pclass'].astype('category')
df['Embarked']=df['Embarked'].astype('category')
df=pd.get_dummies(df)
df

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,22.000000,1,0,7.2500,0,0,1,0,1,0,0,1
1,1,38.000000,1,0,71.2833,1,0,0,1,0,1,0,0
2,1,26.000000,0,0,7.9250,0,0,1,1,0,0,0,1
3,1,35.000000,1,0,53.1000,1,0,0,1,0,0,0,1
4,0,35.000000,0,0,8.0500,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.000000,0,0,13.0000,0,1,0,0,1,0,0,1
887,1,19.000000,0,0,30.0000,1,0,0,1,0,0,0,1
888,0,29.699118,1,2,23.4500,0,0,1,1,0,0,0,1
889,1,26.000000,0,0,30.0000,1,0,0,0,1,1,0,0


In [6]:
x_data=df.iloc[:,1:]
y_data=df.iloc[:,0]
#print(y_data)#인덱스까지 출력됨
x_data=x_data.values
y_data=y_data.values#인덱스 빼고 출력하고 싶을 때
print(x_data)
print(y_data)

[[22.          1.          0.         ...  0.          0.
   1.        ]
 [38.          1.          0.         ...  1.          0.
   0.        ]
 [26.          0.          0.         ...  0.          0.
   1.        ]
 ...
 [29.69911765  1.          2.         ...  0.          0.
   1.        ]
 [26.          0.          0.         ...  1.          0.
   0.        ]
 [32.          0.          0.         ...  0.          1.
   0.        ]]
[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 

In [17]:
x_train, x_test, y_train, y_test = model_selection.train_test_split( #split을 할 때마다 값이 조금씩 달라짐
 x_data, y_data, test_size=0.3)

estimator = LogisticRegression(C=1, max_iter=10000)
estimator.fit(x_train, y_train)

y_predict = estimator.predict(x_train)
score = metrics.accuracy_score(y_train, y_predict) #classification
print('train score: ', score)

y_predict = estimator.predict(x_test)
score = metrics.accuracy_score(y_test, y_predict)
print('test score: ', score)
#train score>test score이 가장 좋은 결과

train score:  0.8154093097913323
test score:  0.7873134328358209


In [18]:
print(x_test[:2])
y_predict=estimator.predict(x_test[:2])
print(y_predict)
for y1, y2 in zip(y_test, y_predict):
    print(y1, y2, y1==y2)

[[16.   0.   1.  39.4  1.   0.   0.   1.   0.   0.   0.   1. ]
 [37.   1.   0.  26.   0.   1.   0.   0.   1.   0.   0.   1. ]]
[1 0]
1 1 True
0 0 True


In [ ]:
#디서트리가 어떤 중요한 일을 했는지 알 수 있기 때문에 중요함
#데이터 슬라이싱, 결측치 채우기, 카테고리 변경
#거의 비슷하게
#슬라이싱과 결측지는 기본이기 때문에 필수로 공부!!!!